<a href="https://colab.research.google.com/github/akimotolab/Policy_Optimization_Tutorial/blob/main/1_policy_optimization_introduction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 方策最適化イントロダクション

この資料は，方策最適化という最適化問題を数値実験を通して理解してもらうことを目的としています．
Gymnasiumと呼ばれるシミュレーション環境を用いて，実際に方策を人手で最適化したり，最適化アルゴリズムを用いて最適化する経験を通して，方策最適化を理解します．
そのため，理論的な説明は大部分割愛してあります．
理論的な部分に興味がある方は，以下の文献などを参考にしてください．

Richard S. Sutton and Andrew G. Barto, Reinforcement Learning: An Introduction, 2nd ed., The MIT Press. (2020) http://incompleteideas.net/book/RLbook2020trimmed.pdf

## Google Colab上でGymnasiumを利用する準備

以下では，必要なパッケージのインストールとインポート，仮想displayの設定を実行しています．
手元の環境でGymnasiumを利用する場合には特に難しいことはありませんが，ノートブックで実行する場合には仮想的なdisplayを設定することが必要になるなど，少し事前準備が必要です．


In [ ]:
# 必要なパッケージのインストール
!apt update
!pip install swig
!apt install xvfb
!pip install pyvirtualdisplay
!pip install gymnasium[box2d]

仮想ディスプレイとGPUの設定（GPUは今回は不要ですが，追々必要になると考えられますので，設定だけしておきます．）

In [ ]:
from pyvirtualdisplay import Display
import torch

# 仮想ディスプレイの設定
_display = Display(visible=False, size=(1400, 900))
_ = _display.start()

# gpuが使用される場合の設定
device = torch.device("cuda" if torch.cuda.is_available() else "cpu" )

必要なパッケージをインポート

In [ ]:
import random
import numpy as np
from scipy.special import softmax
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import animation
import seaborn as sns
import gymnasium as gym
from IPython import display

## 方策最適化とは

まずは方策最適化に触れてみましょう．
ここでは，Cart Pole環境における方策最適化を実際に行ってみます．

この環境では，
カート上のポールを垂直上向きに保ち続けることが目標となります．
この目標を表現するために，角度が-12度から12度の間であれば報酬1が得られるようになっています．
角度がこの範囲から外れたとき，そこで実行が終了されます（terminated フラグが Trueになる）．すなわち，報酬の総和がポールを上向きに保ち続けたステップ数，ということになります．

取れる行動は，カートを右側にpushする（行動 1）か，左側にpushする（行動 0）かの二択になります．
与えられる力は自動的に定まります．

適切な行動を設計するには，現在の状態を観測する必要があります．
観測は以下の4つの情報からなる実数値ベクトルです．

1. cart position
2. cart velocity
3. pole angle
4. pole angular velocity

それぞれの範囲については，以下のURLを確認してください．

https://gymnasium.farama.org/environments/classic_control/cart_pole/

観測状態に応じて行動を決定するルールが方策です．

以下で定義する`rollout`関数は，与えられた方策を用いて，現在の状態を観測し，この観測に基づいて方策が行動選択し，この行動を実際に実行することで次の状態に遷移する，というプロセスを繰り返しています．

`visualize`関数は，`rollout`の結果をアニメーションとして可視化する関数です．

`cumulative_reward`関数は，`rollout`の結果から方策の望ましさを示す指標値を計算します．
大きいほどよい方策ということになります．



In [ ]:
def rollout(envname, policy=None, render=False, seed=None):
    if render:
        env = gym.make(envname, render_mode="rgb_array")
    else:
        env = gym.make(envname)
    history = []
    img = []

    # 乱数の設定
    if seed is not None:
        random.seed(int(seed))
    envseed = random.randint(0, 1000)
    actseed = random.randint(0, 1000)
    observation, info = env.reset(seed=envseed)
    env.action_space.seed(actseed)

    # 可視化用の設定
    if render:
        d = Display()
        d.start()
        img.append(env.render())

    # メインループ（環境とのインタラクション）
    terminated = False
    truncated = False
    while not (terminated or truncated):

        # 行動を選択
        if policy is None:
            action = env.action_space.sample()
        else:
            action = policy(observation)

        # 行動を実行
        next_observation, reward, terminated, truncated, info = env.step(action)
        history.append([observation, action, next_observation, reward, terminated, truncated, info])
        observation = next_observation
        if render:
            display.clear_output(wait=True)
            img.append(env.render())
    env.close()
    return history, img


def visualize(img):
    dpi = 72
    interval = 50
    plt.figure(figsize=(img[0].shape[1]/dpi, img[0].shape[0]/dpi), dpi=dpi)
    patch = plt.imshow(img[0])
    plt.axis=('off')
    animate = lambda i: patch.set_data(img[i])
    ani = animation.FuncAnimation(plt.gcf(), animate, frames=len(img), interval=interval)
    display.display(display.HTML(ani.to_jshtml()))
    plt.close()


def cumulative_reward(history):
    return sum(hist[3] for hist in history)

上のコードにおいて重要な箇所を解説します．

observation, info = env.reset(seed=...) : 状態を初期化しています．observationが初期の状態，infoは環境固有の情報を含んでいます．初期状態はランダムにきまるため，そのシードがseed=..で指定されています．このシードが固定されていると，毎回同じ様に初期化されることになります．

action = policy(observation) : 現在の状態の観測値 observation に従って，方策により行動 action を選択します．

observation, reward, terminated, truncated, info = env.step(action) : 行動 action を実行し，次の状態の観測値 observation，即時報酬 reward，終了状態に達したことを表すフラグ terminated，最大ステップ数に達したことを表すフラグtruncated，その他の情報 infoを返します．

## ランダム方策を用いてインタラクションしてみる

それではまず，取りうる行動の範囲からランダムに行動を選択するランダム方策（観測した状態によらず，取り得る行動の値の範囲から一様ランダムな行動をとる方策）を用いて，実際に環境とインタラクションしてみましょう．
先程定義した`rollout`関数では，`policy`を引数に与えない場合，ランダム方策を用いるように定義されています．
それでは以下を実行してみましょう．

すぐに倒れてしまう様子が確認できると思います．
ランダムに行動を選択しているということは，ポールの状態を考えずにサイコロで行動を決定しているということになりますから，当然ですね．


In [ ]:
envname = "CartPole-v1"
history, img = rollout(envname, render=True)  # 可視化する場合には render = True
visualize(img)  # 可視化
print(cumulative_reward(history))  # 累積報酬

## 方策を手作りしてみる

ここでは，自分で方策を作成してみましょう．

簡単に思いつく方法としては，

- ポールが右側に倒れていれば（角度が正の値）ならば，右側にカートを押す（行動 1）
- ポールが左側に倒れていれば（角度が負の値）ならば，左側にカートを押す（行動 0）

かと思います．
これをまずは試してみましょう．

`rollout`では，方策は観測`observation`を受け取って行動を返すcallableであることを想定しています．
ここでは関数として方策を定義します．（後にクラスとして定義します．）

In [ ]:
def manual_policy(observation):
    # observation[2] が角度．負は左側に傾いている
    if observation[2] < 0:
        return 0
    else:
        return 1

この方策に従って行動した場合，どの程度ポールが立ち続けていられるのか，確認してみましょう．
最後にプリントされる`cumulative_reward`の値がここではポールを指定された角度に保ち続けていられたステップ数です．

In [ ]:
envname = "CartPole-v1"
history, img = rollout(envname, policy=manual_policy, render=True)  # 可視化する場合には render = True
visualize(img)  # 可視化
print(cumulative_reward(history))  # 累積報酬

ランダム行動の場合よりは少し長くポールを保てる用になっているかと思います．
ただし，ポールの振れ幅がどんどん大きくなってしまっていることが観察されるはずです．
これは，ポールの速度に関する情報や，カートの速度の情報を無視していることが原因と考えられます．

そこで，ポールの速度も用いて行動選択をしてみましょう．
ここでは，まずポールの角度を用いて条件分け，その後ポールの角速度を用いて条件分け，という手続きで行動選択することを考えましょう．
上のケースと同じように，ポールが左に傾いていれば，左側に押すことが自然です．
その際，角速度が負であるか，もしくは正の小さな値であれば，左側に押すことで，角速度を正側に大きくすることになるため，やはり左側に押すことが適切であるように思います．
他方，ポールが左に傾いていても，角速度が正の大きな値であるならば，押さなくても自然にポールは垂直方向に変化します．
逆に，ポールの角速度を小さくするほうが，安定して垂直に保てると考えられます．
右側に傾いている場合にも，上と同様に考えられます．

そこで，下に定義するような方策を考えてみましょう．

1. $\theta < 0$かつ$\dot{\theta} < U$ならば左に押す（行動0）
2. $\theta < 0$かつ$\dot{\theta} \geq U$ならば右に押す（行動1）
1. $\theta \geq 0$かつ$\dot{\theta} > L$ならば右に押す（行動1）
2. $\theta \geq 0$かつ$\dot{\theta} \leq L$ならば左に押す（行動0）

ここで，$U$や$L$はパラメータです．
上の議論から，$U > 0$，$L < 0$となることが自然です．
なお，`manual_policy`は，$U = \infty$，$L = -\infty$であるケースに該当していることがわかるかと思います．

以下のコードにおいて，$L$や$U$を変更し，長い間（最大500ステップ）ポールを垂直に保つように試行錯誤してみましょう．

In [ ]:
import numpy as np
def manual_policy2(observation):
    U = np.infty
    L = - np.infty
    if observation[2] < 0:
        if observation[3] < U:
            return 0
        else:
            return 1
    else:
        if observation[3] > L:
            return 1
        else:
            return 0

試行錯誤をしている間は，動画は作成する必要ありません．
`cumulative_reward` の返り値を最大（500）にするように，$L$と$U$を変更しましょう．
動作を確認したいときは，コメントアウトを外して動画を作成しましょう．

In [ ]:
envname = "CartPole-v1"
history, img = rollout(envname, policy=manual_policy2, render=True)  # 可視化する場合には render = True
# visualize(img)  # 可視化
print(cumulative_reward(history))  # 累積報酬

以下，良い方策を示しているのでわざと空白を入れています．
自分で試行錯誤してから先に進みましょう．

.

.

.

.

.

.

.

.

.

.

.

.

.

.

.

.

.

.

.

.

.

.

.

.

.

.

.

.

.

.

.

.

.

.

.

.

.

.

.

.

.

.

.

.

.

.

.

.

.

.

.

.

.

.

.

.

.



$L = -0.1$，$U = 0.1$などと設定すると，500ステップずっとポールを保ち続けることができます．

In [ ]:
import numpy as np
def manual_policy3(observation):
    U = 0.1
    L = - 0.1
    if observation[2] < 0:
        if observation[3] < U:
            return 0
        else:
            return 1
    else:
        if observation[3] > L:
            return 1
        else:
            return 0

In [ ]:
envname = "CartPole-v1"
history, img = rollout(envname, policy=manual_policy3, render=True)  # 可視化する場合には render = True
visualize(img)  # 可視化
print(cumulative_reward(history))  # 累積報酬

さて，今実際に行ってみたように，$L$や$U$を変えるとそれによって方策が定まり，その結果得られる累積報酬が変化しました．
このように，累積報酬を最大にするような方策を見つけ出すことこそが方策最適化です．
$L$や$U$は方策を表現するためのパラメータです．
基本的には，方策は何らかの方法（ここでは決定木と呼ばれるもの）で表現され，それはパラメータを持つことになります．
方策最適化は，このパラメータを最適化することで，得られる累積報酬を最大化します．

# ブラックボックス最適化アルゴリズムを用いた方策の直接探索
上に見た CartPole 環境では行動が $\{0, 1\}$ の離散値であり，少し考えればうまく動く単純な方策を人が容易に設計できました．
一般的にはそう簡単には良い方策を人手で設計することはできません．
例えば，行動が連続値である場合には，そう簡単にうまく行かないかもしれません．
その場合，方策最適化問題を，最適化アルゴリズムを用いて解く方法が考えられます．
ここではその一例を見てみましょう．

## 使用環境１：Pendulum（連続状態，連続行動）
ここでは，Pendulum環境を用いて方策最適化問題の例を見ていきます．
環境については，以下のURLで説明されています．

https://gymnasium.farama.org/environments/classic_control/pendulum/

ランダムに初期化されたpendulumの角度を，垂直上向きに振り上げて保ち続けることが目標となります．
観測状態は角度$\theta$に対して，

1. $\cos(\theta) \in [-1, 1]$
2. $\sin(\theta) \in [-1, 1]$
3. $\dot\theta \in [-8, 8]$

となっており，行動はpendulumに対するトルクです．
以下，ランダム行動を取る場合の振る舞いです．
報酬は負の値であり，各ステップ$[-16.2736044, 0]$の報酬が与えられます．
200ステップ分の累積が大きくなる，すなわち$0$に近いほど，良い方策であると言えます．

In [ ]:
envname = "Pendulum-v1"
history, img = rollout(envname, render=True)  # 可視化する場合には render = True
visualize(img)  # 可視化
print(cumulative_reward(history))  # 累積報酬

ランダム行動だと-1000を下回る程度の累積報酬になってしまっていることがわかるかと思います．

## 線形モデルによる方策の表現　（連続行動，決定方策）

それでは方策を最適化することを考えていきましょう．
まず，最適化対象となる方策をモデル化する必要があります．
ここでは簡単のため，線形モデルを考えましょう．

観測 `observation` は三次元のベクトルであり，値は$[-1, 1] \times [-1, 1] \times [-8, 8]$です．
簡単のため，これをベクトル$\mathbf{x} = (x_1, x_2, x_3)$ と書きます．
また，便宜上，観測の先頭に定数$1$を加えたベクトルを$\bar{\mathbf{x}} = [1, \mathbf{x}^\mathrm{T}]^\mathrm{T}$と書くことにします．
行動$a$は一次元の値であるため，これを
$$
a = 2 \tanh( \mathbf{w}^\mathrm{T} \bar{\mathbf{x}}) = 2 \tanh( w_0 + w_1 x_1 + w_2 x_2 + w_3 x_3 )
$$
と決定する方策を考えましょう．
このとき，最適化対象は四次元ベクトル$\mathbf{w} = (w_0, w_1, w_2, w_3)$です．
行動の取り得る範囲が$[-2, 2]$なので，線形関数の出力をtanhで$[-1, 1]$に補正したのち，2倍しています．






In [ ]:
class LinearPolicy:
    def __init__(self, weights):
        self.weights = weights
    def __call__(self, observation):
        return np.array([2 * np.tanh(self.weights[0] + np.dot(observation, self.weights[1:4]))])

ここでは方策をクラスとして定義しています．__call__を定義しておくことで，このクラスのインスタンスをあたかも関数であるかのように呼び出すことができます．関数でなくクラスとして定義しておく理由は，パラメータであるweightsをあとから変更するためです．

## 目的関数の定義

最適化問題としてアプローチするために，目的関数を定義します．
ここでの目的は累積報酬を最大化することです．
そのため，目的関数$J(\mathbf{w})$を，パラメータ$\mathbf{w}$の線形方策を用いて得られる累積報酬としましょう．

この目的関数を$\mathbf{w}$の関数として書き下すことは，状態遷移確率や報酬関数を知らない限りできません．
そこで，与えられたパラメータ$\mathbf{w}$の線形方策を用い，実際に環境とのインタラクションを通して目的関数の値を計算します．

以下で，この目的関数の値を計算する関数を定義しています．
本来，方策最適化の目的は，任意の初期状態から得られる累積報酬を最大にするような方策を獲得することですが，ここでは簡単のため，初期状態は固定（環境を特定のシードで初期化）することを考えます．
中身は，これまでに実施してきたメインループそのものとなっています．
また，最適化アルゴリズムは多くの場合，最小化問題を想定しているので，目的関数を$-1$倍することにします．
最後に$10^{-10} \times \|\mathbf{w}\|^2$を目的関数に足しています．
これは，重みパラメータが発散していくことを防ぐ目的のために追加されています．

In [ ]:
envname = "Pendulum-v1"
dim_state = 3
dim_action = 1
dim_x = dim_state + 1


def objective1(x):
    policy = LinearPolicy(x)
    history, img = rollout(envname, policy, seed=100)
    return -cumulative_reward(history) + 1e-10 * np.dot(x, x)

適当な値を入れてみましょう．
シードを固定しているので，同じパラメータを入力すれば，何度計算しても同じ値が帰ってくるはずです．

In [ ]:
objective1(np.array([0, 0, 0, 0]))

## (1+1)-CMA-ES による最適化

ブラックボックス最適化アプローチである，(1+1)-CMA-ESを用いて最適化してみましょう．
多くの数値最適化法では，目的関数の勾配を利用します．
しかし，今回の問題では，目的関数の勾配を得ることが自明でないと容易にわかるかと思います．
(1+1)-CMA-ESは，勾配を使わず，目的関数の大小関係のみを用いて探索する方法であるため，非常に適用範囲の広いアプローチです．

(1+1)-CMA-ESは，正規分布からの解生成を繰り返す山登り法です．
正規分布の中心が現在の解です．
生成された解が現在の解以上に良い目的関数値をとるならば，正規分布の中心をそちらに移動します．
さもなくば動きません．
効率的に解を探索するために，良い解が生成されやすいように，正規分布の共分散行列も同時に更新していきます．
良い解の尤度を高くするような（最尤推定のような）更新を行っていると解釈できます．


In [ ]:
class Cmaes:
    """(1+1)-Active-CMA-ES [Arnold 2010] with Simplified 1/5th Success Rule"""

    def __init__(self, init_fx, init_x, init_s):
        self.dim = len(init_x)

        self.aup = np.exp(2.0 / (2.0 + self.dim))
        self.adown = self.aup ** (-0.25)
        self.pthresh = 0.44
        self.cp = 1.0 / 12.0
        self.cc = 2.0 / (self.dim + 2)
        self.ccovp = 2.0 / (self.dim**2 + 6)
        self.ccovm = 0.4 / (self.dim**1.6 + 1)

        self.x = np.array(init_x, copy=True)
        self.xc = np.zeros(self.dim)
        self.yc = np.zeros(self.dim)
        self.zc = np.zeros(self.dim)
        self.s = init_s
        self.psucc = 0.5
        self.fhist = [init_fx]
        self.p = np.zeros(self.dim)
        self.chol = np.eye(self.dim)
        self.cholinv = np.eye(self.dim)

    def ask(self):
        self.zc = np.random.randn(self.dim)
        self.yc = np.dot(self.chol, self.zc)
        self.xc = self.x + self.s * self.yc
        return self.xc

    def tell(self, fx):
        if fx <= self.fhist[0]:
            self.fhist = [fx] + self.fhist[:min(4, len(self.fhist))]
            self.x = self.xc
            self.s *= self.aup
            self.psucc = (1.0 - self.cp) * self.psucc + self.cp
            if self.psucc > self.pthresh:
                self.p = (1.0 - self.cc) * self.p
                d = self.ccovp * (1.0 - self.cc * (2.0 - self.cc))
            else:
                self.p = (1.0 - self.cc) * self.p + np.sqrt(self.cc * (2.0 - self.cc)) * self.yc
                d = self.ccovp
            w = np.dot(self.cholinv, self.p)
            wnorm2 = np.dot(w, w)
            if wnorm2 > 1e-6:
                a = np.sqrt(1.0 - d)
                b = np.sqrt(1.0 - d) * (np.sqrt(1.0 + self.ccovp * wnorm2 / (1.0 - d)) - 1.0) / wnorm2
                self.chol = a * self.chol + b * np.outer(self.p, w)
                self.cholinv = self.cholinv / a - (b / (a**2 + a * b * wnorm2)) * np.outer(w, np.dot(w, self.cholinv))
        else:
            self.s *= self.adown
            self.psucc = (1.0 - self.cp) * self.psucc
            if len(self.fhist) > 4 and fx > self.fhist[4] and self.psucc <= self.pthresh:
                znorm2 = np.dot(self.zc, self.zc)
                ccov = self.ccovm if 1.0 >= self.ccovm * (2.0 * znorm2 - 1.0) else 1.0 / (2.0 * znorm2 - 1.0)
                a = np.sqrt(1.0 + ccov)
                b = np.sqrt(1.0 + ccov) * (np.sqrt(1.0 - ccov * znorm2 / (1.0 + ccov)) - 1) / znorm2
                self.chol = a * self.chol + b * np.outer(self.yc, self.zc)
                self.cholinv = self.cholinv / a - (b / (a**2 + a * b * znorm2)) * np.outer(self.zc, np.dot(self.zc, self.cholinv))

それではESオブジェクトを生成し，最適化してみましょう．

In [ ]:
# 初期分布の設計
sigma = 10.0
mean = sigma* np.random.randn(4)
fmean = objective1(mean)

# 履歴
f_hist = np.zeros(1000)

# オブジェクトの作成
es1 = Cmaes(fmean, mean, sigma)

以下が実行コードです．もし十分に収束していないと判断される場合，実行を繰り返せば続きからの最適化が実行されます．
実行後に探索過程で得られた解の評価値をプロットしています．

In [ ]:
# 実行
for t in range(len(f_hist)):
    w = es1.ask()  # 新しい解を生成
    f_hist[t] = objective1(w)  # 新しい解を評価
    es1.tell(f_hist[t])  # 分布を更新
    if (t+1) % 10 == 0:
        print(t+1, np.min(f_hist[:t+1]), es1.s)
    if t >= 300 and np.min(f_hist[:t+1-300]) == np.min(f_hist[:t+1]):
        f_hist = f_hist[:t+1]
        break

plt.plot(f_hist)

最後に振る舞いを可視化してみます．

In [ ]:
envname = "Pendulum-v1"
policy = LinearPolicy(es1.x)
history, img = rollout(envname, policy=policy, render=True, seed=100)  # 可視化する場合には render = True
visualize(img)  # 可視化
print(cumulative_reward(history))  # 累積報酬

Pendulumタスクの場合，優れた方策であれば，累積報酬が-100程度までよくなります．
(1+1)-ESは乱数を用いた最適化法ですので，試行毎に得られる解が変わりますが，概ねそこまでの累積報酬は得られないような方策が獲得されていると思います．

## 方策表現の再検討

線形方策は単純なため，理解しやすいですが，表現能力が不足していました．
そこで，これを一般化したニューラルネットワークによる方策の表現方法を考えましょう．
ここでは一例として，`softmax`関数を用いた２層のニューラルネットワークを考えることにします．

### 線形方策の表現能力

まず，上で用いた線形方策における表現能力について，簡単に見ておきましょう．

さて，線形方策は，観測された状態（$\bar{\mathbf{x}}$）と重みベクトル（$\mathbf{w}$）の内積をとり，その値を$\tanh$により取り得る値に変換しています．
行動の正負だけに着目するならば，これは内積の値で一意にさだまりますから，状態空間を重みベクトル$\mathbf{w}$で定義される超平面で分けたとき，片側では正の行動を出力し，もう片側では負の行動を出力するようになります．

Pendulum環境の場合，仮にpendulumがほとんど垂直上向きであるならば，$\sin(\theta)$の符号だけを見て制御すれば，おおよその制御は可能と考えられますが，振り上げ動作が必要な場合にはこのような単純な制御では振り上げおよび安定化の両方を実現することは困難と思われます．

このような場合，超平面で線引するだけでなく，領域ごとに異なる行動を取れるような表現能力の高い非線形な方策が必要となります．

### ニューラルネットワークによる方策表現
ここでは，行動を以下のように決定するニューラルネットワークを考えます．
$$
a = \mathbf{V} \cdot \mathrm{softmax}\left( \mathbf{W} \cdot \mathbf{x} + \mathbf{b}\right)
$$
一般的な説明をするために，状態の次元数を$N_s$（ここでは$N_s = 3$），行動に次元数を$N_a$（ここでは$N_a = 1$）と書くことにします．
最適化対象のパラメータは$\mathbf{b} \in R^K$，$\mathbf{W} \in R^{K \times N_s}$，$\mathbf{V} \in R^{N_a \times K}$の３つであり，合計$K (1 + N_s + N_a)$個のパラメータからなります．

この方策の意味を少し考えておきましょう．
$\mathrm{softmax}$は入力ベクトル$\mathbf{y} = [y_1, \dots, y_n]$に対して
$$
\mathrm{softmax}(\mathbf{y}) = \frac{[\exp(y_1), \dots, \exp(y_n)]}{\sum_{i=1}^{n} \exp(y_i)}
$$
を計算する関数です．
出力は必ず正で和が$1$になるようなベクトルです．
名前の通り，$\max$を近似するような関数です．
上の方策の式において，$\mathrm{softmax}$の出力を$\mathbf{z}$などと書けば，行動は$\mathbf{V} = [\mathbf{v}_1 \dots \mathbf{v}_K]$ の列ベクトル$\mathbf{v}_k$の線形補間
$$
\mathbf{a} = \mathbf{V} \cdot \mathbf{z} = \sum_{k=1}^{K} z_k \mathbf{v}_k
$$
で決まります．
$\mathrm{softmax}$関数が$\max$の近似であることを考慮すれば，

* $\mathbf{V} = [\mathbf{v}_1 \dots \mathbf{v}_K]$ が取る行動の候補$K$個を定める部分

* $ \mathbf{W} \cdot \mathbf{x} + \mathbf{b}$ がK個のうちどの行動を選択するかを決める部分

であることがわかります．
これは，状態空間をボロノイ分割し（分割方法は$\mathbf{W}$と$\mathbf{b}$を最適化することで変化する），各領域で選択する行動が$\mathbf{v}_k$で定まる，と解釈されます．
実際には，$\max$ではなく$\mathrm{softmax}$であるため，その間の区間を線形補間していることになります．

なお，上記説明からは省略しましたが，出力は定義域に収まるよう，$\tanh$を用いて出力範囲を変換しています．


In [ ]:
envname = "Pendulum-v1"
dim_state = 3
dim_action = 1
K = 20
N = (dim_state + dim_action + 1) * K


class NNPolicy:
    def __init__(self, weights, K=K):
        Na = dim_action
        Ns = dim_state
        self.B = weights[:K]
        self.W = weights[K:K*Ns+K].reshape((K, Ns))
        self.V = weights[K*Ns+K:K*Ns+K+K*Na].reshape((Na, K))

    def __call__(self, observation):
        z = np.dot(observation, self.W.T)
        z += self.B
        z = softmax(z)
        z = np.dot(z, self.V.T)
        return 2 * np.tanh(z)

In [ ]:
def objective2(x):
    policy = NNPolicy(x)
    history, img = rollout(envname, policy, seed=100)
    return -cumulative_reward(history) + 1e-10 * np.dot(x, x)

sigma = 10.0
mean = sigma * np.random.randn(N)
f_hist = np.zeros(1000)
fmean = objective2(mean)
es2 = Cmaes(fmean, mean, sigma)

In [ ]:
for t in range(len(f_hist)):
    w = es2.ask()
    f_hist[t] = objective2(w)
    es2.tell(f_hist[t])
    if (t+1) % 10 == 0:
        print(t+1, np.min(f_hist[:t+1]), es2.s)
    if t >= 300 and np.min(f_hist[:t+1-300]) == np.min(f_hist[:t+1]):
        f_hist = f_hist[:t+1]
        break

plt.plot(f_hist)

ニューラルネットワークを用いた方策の場合，数回最適化を実施してみれば，200程度の目的関数値を達成する方策が得られることと思います．
- 生成されるグラフは，最適化中に生成した解（方策パラメータ）の目的関数値を示しています．
- 出力される数値は，左からイテレーション数，現在得られている解の目的関数値，ESの解探索の標準偏差パラメータ$\sigma$の値
- 収束していない（$\sigma$の値が十分に小さくない(例えば>1e-2)）場合には，for-loop部分だけもう一度実行してみましょう．途中から最適化が追加で行われます．
- 新しい最適化を実行する（新しい方策パラメータの初期値から探索する）場合には，Cmaesインスタンスももう一度作り直しましょう．


獲得された方策の振る舞いを確認してみましょう．

In [ ]:
envname = "Pendulum-v1"
policy = NNPolicy(es2.x)
history, img = rollout(envname, policy=policy, render=True, seed=100)
print(cumulative_reward(history))
visualize(img)

以上より，十分な表現能力をもつ方策モデル選択することが重要であることがわかってもらえたかと思います．

## 汎化性能の評価

さて，上の最適化の際には，pendulumの初期状態を固定して方策パラメータを最適化しました．
実際には，他の初期状態から始めたとしても良い方策となって欲しいので，初期値がランダムに決まるケースでの累積報酬の期待値を計算してみましょう．
期待値を計算するといっても，解析的に計算できるわけではないので，複数回ランダムな初期値から実行し，その平均を計算してみます．

以下では，与えた回数だけ累積報酬を計算（エピソードを回し）し，その結果（配列）を計算しています．

In [ ]:
envname = "Pendulum-v1"
policy = NNPolicy(es2.x)  # 最適化によって得られた方策
return_array = np.zeros(50)
for i in range(len(return_array)):
    history, img = rollout(envname, policy=policy, render=False)
    return_array[i] = cumulative_reward(history)

経験分布関数をプロットしてみましょう．
ここで表示しているものは，横軸（累積報酬の -1倍の値 = 目的関数の値）より小さな目的関数値が得られた割合です．
例えば，縦軸が0.5のときの横軸の値が，得られた目的関数値の中央値となります．

In [ ]:
fig, ax = plt.subplots()
sns.ecdfplot(data=-return_array, ax=ax)
ax.set_xlim(0, 1800)
plt.grid()

この結果を見ると，特定の初期値からの方策を最適化しただけでは，他の初期値から始めた場合に必ずしも適切な行動が取れていないことがわかります．
目的関数が特定の初期値からの方策の性能を評価しているため，他の初期値から始めた場合の性能を考慮していないことが一因です．

## 目的関数の再検討

最適化法は目的関数の値を良くすることだけを目標に最適なパラメータを求めていくため，使用者が望ましいと考える方策の性質を実現するためには，そのような性質をもつ方策に対してインセンティブを与えるような目的関数が設計されていることが必要です．
今回の場合，様々な初期値から始めたとしても高速にpendulumを振り上げて安定化させることが目的ですから，これを評価するような目的関数を設計する必要があります．

### パーセンタイルの最適化
そこで，目的関数を，初期値を変えて10回方策を評価した際の90パーセンタイルとしてみましょう．
このようにすることで，9割くらいの初期状態に対しては良い報酬が得られるような方策を獲得できると期待されます．
なお，目的関数の内部で計算するエピソード数が増えるほど，計算時間が増えていきますので，注意してください．

In [ ]:
import numpy as np

def objective3(x):
    envname = "Pendulum-v1"
    policy = NNPolicy(x)
    seed_array = np.arange(100, 1100, 100)
    return_array = np.zeros(len(seed_array))
    for i, seed in enumerate(seed_array):
        history, img = rollout(envname, policy=policy, render=False, seed=seed)
        return_array[i] = cumulative_reward(history)
    return np.percentile(- return_array, 90) + 1e-10 * np.dot(x, x)

sigma = 10
mean = sigma * np.random.randn(N)
f_hist = np.empty(1000)
f_hist[0] = objective3(mean)
es3 = Cmaes(f_hist[0], mean, sigma)

In [ ]:
for t in range(len(f_hist)):
    w = es3.ask()
    f_hist[t] = objective3(w)
    es3.tell(f_hist[t])
    if (t+1) % 10 == 0:
        print(t+1, np.min(f_hist[:t+1]), es3.s)
    if t >= 300 and np.min(f_hist[:t+1-300]) == np.min(f_hist[:t+1]):
        f_hist = f_hist[:t+1]
        break

plt.plot(f_hist)

この目的関数（90パーセンタイル）の下でも，うまくいくと200程度の目的関数値が得られます．
ただ，一度の目的関数評価に複数回のエピソードを実行しているため，最適化実行時間は長くなります．



目的関数は上においてもシードを固定した複数試行から計算されています．
これは，目的関数が確率的に変動してしまうと最適化が困難になるため，簡単化するための工夫です．
ただし，この場合，特定のシード（特定の初期状態）にオーバーフィットしてしまう可能性がありますので，様々な環境で評価してみましょう．

In [ ]:
envname = "Pendulum-v1"
policy = NNPolicy(es3.x)
return_array = np.zeros(50)
for i in range(len(return_array)):
    history, img = rollout(envname, policy=policy, render=False)
    return_array[i] = cumulative_reward(history)

In [ ]:
fig, ax = plt.subplots()
sns.ecdfplot(data=-return_array, ax=ax)
ax.set_xlim(0, 1800)
plt.grid()

こちらも，試行によって結果は変わるかと思いますが，うまく行くと目的関数値の90パーセンタイルが200付近の値をとるような方策が獲得できます．
すなわち，10回に9回は非常にうまくpendulumを垂直上向きに保つことができるような方策が得られるということです．
ただし，最適化している目的関数で計算しているパーセンタイルはあくまでシードを固定した推定値ですから，多少はズレがあると思います．
また，ワーストケースなどを見ると，最適化時に観測された目的関数値よりも大きく悪い結果になっていることが確認できると思います．
このように，最適化法は，与えられた目的関数が計算する値については最適化するものの，そこに反映されていない人が暗黙に期待するような振る舞いを獲得してくれるわけではありません．

以下で振る舞いを確認してみましょう．
環境を初期化する際に乱数のシードを指定しなければ，毎回異なる初期値に初期化されます．
以下を何度か繰り返し，うまくいく試行，いかない試行の振る舞いを見てみましょう．

In [ ]:
envname = "Pendulum-v1"
policy = NNPolicy(es3.x)
history, img = rollout(envname, policy=policy, render=True)
print(cumulative_reward(history))
visualize(img)

## 使用環境２：Lunar Lander（連続状態，離散行動）

もう一つ，別の環境での例を見てみましょう．
次に扱うのは Lunar Lander 環境です．

https://gymnasium.farama.org/environments/box2d/lunar_lander/

この環境では，エージェントが$8$次元のセンサー情報（連続値）を状態として観測し，$4$つの行動から一つを選択します．
その行動は

0. 何もしない
1. 左向きのエンジンを蒸す
2. メインエンジンを蒸す
3. 右向きのエンジンを蒸す

というものです．
すなわち，先程とは異なり，行動が離散値（カテゴリカル）を取ります．

まずは以下を実行し，ランダム行動を取る場合の振る舞いを確認してみましょう．

In [ ]:
history, img = rollout("LunarLander-v2", render=True)
cumulative_reward(history)
visualize(img)

行動が連続か離散かという違いの他に，もう一点，重要な違いが PendulumとLunar Landerの間に存在します．

* Pendulum 環境やCart-Pole環境では，予め定められた回数（200回, 500回）のインタラクションを実行すると， フラグ truncated が Trueとなり，状態が初期化されます．

* Lunar Lander環境では，着陸に成功 or 着地失敗など，特定の状態になると，成功 or 失敗という認識になり，フラグterminated がTrueとなり，状態が初期化されます．

前者の場合，例えば目的関数を計算する都合上，無限に長いインタラクション数を取ることはできないため，強制的に終了させているとみなせます．
他方，後者の場合には，明確に終了となる状態が定義されており，「一度の試行」という概念が定義されているようなタスクであると言えます．
後者のようなタスクを「エピソディックタスク」と呼びます．
この二つの違いは，強化学習を行う上では重要な違い（使用できる方法に違いが生じる）であるため，認識しておきましょう．
ただ，ブラックボックス最適化として考える場合には，大きな差はありません．


### Softmax方策

行動が離散ですので，先程とは異なる方策が必要になります．
行動が4種類であるということは，状態空間を4分割し，各分割では一つの行動を選択るという方法が考えられます．
これは，教師あり学習で言うところの分類タスクに該当します．
そこで，分類タスクによく使われるSoftmaxを用いた方策を利用しましょう．
これは，$4$次元出力の線形モデルに対し，softmax関数を適用し，その結果として得られるベクトルを確率ベクトルとして行動を選択する，という確率的方策になります．

ここでも違いが出てきたので紹介しておきます．
Pendulum環境で用いていた方策はいずれも状態が与えられると行動が一意に定まる「決定的方策」でした．
他方，ここで用いている方策は，状態が与えられるとそれによって得られる条件付き確率に従って行動を選択する「確率的方策」になります．
強化学習の場合，良い行動を探索するために確率的方策が必要になる場合があります．
一方，ブラックボックス最適化として扱う場合には，探索は探索法が担っているので，確率的方策を使用することは必要ではありません．
ここでは，確率的方策の紹介と，分類タスクとの関連を紹介するために，利用しています．

Softmax方策において各行動を取る確率は以下のように定義されます．
$$
\pi(a=k \mid s) = \frac{ \exp( \mathbf{w}_k^\mathrm{T} s + b_k) }{ \sum_{i=1}^{4} \exp( \mathbf{w}_i^\mathrm{T} s + b_i) }
$$
ここで，$\mathbf{w}_1 = \mathbf{0}$と$b_1 = 0$はいずれも固定とします．
そのようにしても，表現能力は損なわれません．
逆に，全てのパラメータを変動できるようにした場合，方策がオーバーパラメタライズされた状況となり，「パラメータを変えても表現される方策が変わらず，その結果目的関数が変化しない」ような領域が無数に存在することになり，これが最適化を難しくすることがあります．
結果として，この方策はパラメータ$\theta = (\mathbf{w}_2, \mathbf{w}_3, \mathbf{w}_4, b_2, b_3, b_4)$で表現されることになります．これが最適化の対象となります．


In [ ]:
envname = "LunarLander-v2"
dim_state = 8
num_action = 4
N = (1 + dim_state) * (num_action - 1)

class SoftmaxPolicy:
    def __init__(self, x):
        self.w = np.zeros((num_action, dim_state))
        self.b = np.zeros(num_action)
        self.b[1:] = x[:num_action-1]
        self.w[1:, :] = x[num_action-1:].reshape((num_action - 1, dim_state))

    def __call__(self, observation):
        logit = np.dot(self.w, observation) + self.b
        prob = softmax(logit)
        return random.choices(list(range(len(prob))), prob)[0]

それでは，このSoftmax方策を(1+1)-ESを用いて最適化してみましょう．
目的関数は特定のシード（地形に影響）のもとでの累積報酬の$-1$倍とします．
今回は，方策が確率的であるため，目的関数の値も確率的となります．

In [ ]:
def objective4(x):
    policy = SoftmaxPolicy(x)
    history, img = rollout(envname, policy, seed=100)
    return -cumulative_reward(history) + 1e-10 * np.dot(x, x)


sigma = 10.0
mean = sigma * np.random.randn(N)
f_hist = np.empty(1000)
f_hist[0] = objective4(mean)
es4 = Cmaes(f_hist[0], mean, sigma)

In [ ]:
for t in range(len(f_hist)):
    w = es4.ask()
    f_hist[t] = objective4(w)
    es4.tell(f_hist[t])
    if (t+1) % 10 == 0:
        print(t+1, np.min(f_hist[:t+1]), es4.s)
    if t >= 300 and np.min(f_hist[:t+1-300]) == np.min(f_hist[:t+1]):
        f_hist = f_hist[:t+1]
        break

plt.plot(f_hist)

汎化性能を評価するために，経験分布関数を表示してみましょう．
Pendulumで見た通り，最適化時に特定のシードを用いている（ここでは地形に対応）ため，これが変わるとうまく行かないことが見て取れます．

In [ ]:
envname = "LunarLander-v2"
policy = SoftmaxPolicy(es4.x)
return_array = np.zeros(50)
for i in range(len(return_array)):
    history, img = rollout(envname, policy=policy, render=False)
    return_array[i] = cumulative_reward(history)

In [ ]:
fig, ax = plt.subplots()
sns.ecdfplot(data=-return_array, ax=ax)
ax.set_xlim(-400, 400)
plt.grid()

振る舞いを可視化してみましょう．seed=100は最適化時に利用している環境であり，それ以外の場合と比較してうまく制御されていることが見て取れるかと思います．

In [ ]:
history, img = rollout(envname, policy=policy, render=True, seed=100)
print(cumulative_reward(history))
visualize(img)

In [ ]:
history, img = rollout(envname, policy=policy, render=True)
print(cumulative_reward(history))
visualize(img)

Softmax方策は表現能力が不十分なため，Pendulumにおいて用いたニューラルネットワーク方策を利用してみましょう．
出力が確率ベクトルとなるように，$\tanh$の代わりにsoftmax関数を出力にかけ，その結果を行動選択確率として行動を選択します．

In [ ]:
envname = "LunarLander-v2"
dim_state = 8
num_action = 4
K = 20
N = (dim_state + num_action + 1) * K

class SoftmaxNNPolicy:
    def __init__(self, weights, K=K):
        Na = num_action
        Ns = dim_state
        self.B = weights[:K]
        self.W = weights[K:K*Ns+K].reshape((K, Ns))
        self.V = weights[K*Ns+K:K*Ns+K+K*Na].reshape((Na, K))

    def __call__(self, observation):
        z = np.dot(observation, self.W.T)
        z += self.B
        z = softmax(z)
        z = np.dot(z, self.V.T)
        prob = softmax(z)
        return random.choices(list(range(len(prob))), prob)[0]

In [ ]:
def objective5(x):
    policy = SoftmaxNNPolicy(x)
    history, img = rollout(envname, policy, seed=100)
    return -cumulative_reward(history) + 1e-10 * np.dot(x, x)


sigma = 10
mean = sigma * np.random.randn(N)
f_hist = np.empty(1000)
f_hist[0] = objective5(mean)
es5 = Cmaes(f_hist[0], mean, sigma)

In [ ]:
for t in range(len(f_hist)):
    w = es5.ask()
    f_hist[t] = objective5(w)
    es5.tell(f_hist[t])
    if (t+1) % 10 == 0:
        print(t+1, np.min(f_hist[:t+1]), es5.s)
    if t >= 300 and np.min(f_hist[:t+1-300]) == np.min(f_hist[:t+1]):
        f_hist = f_hist[:t+1]
        break

plt.plot(f_hist)

振る舞いを見てみましょう．

In [ ]:
policy = SoftmaxNNPolicy(es5.x)
history, img = rollout(envname, policy=policy, render=True, seed=100)
print(cumulative_reward(history))
visualize(img)

In [ ]:
policy = SoftmaxNNPolicy(es5.x)
history, img = rollout(envname, policy=policy, render=True)
print(cumulative_reward(history))
visualize(img)

経験分布関数も確認しておきましょう．
やはり，方策を変えるだけでは汎化性能は高くなりません．
目的関数をパーセンタイルにするなどの工夫が必要です．

In [ ]:
envname = "LunarLander-v2"
policy = SoftmaxNNPolicy(es5.x)
return_array = np.zeros(50)
for i in range(len(return_array)):
    history, img = rollout(envname, policy=policy, render=False)
    return_array[i] = cumulative_reward(history)

In [ ]:
fig, ax = plt.subplots()
sns.ecdfplot(data=-return_array, ax=ax)
ax.set_xlim(-400, 400)
plt.grid()

## この章のまとめ

ここでは方策最適化をブラックボックス最適化問題として捉え，(1+1)-CMA-ES を用いて実際に最適化してみました．
目的関数の設計の仕方，方策の選び方，などによって，最終的に得られる方策に違いが見られる様子が確認できたと思います．
これは，(1+1)-CMA-ESによらず，強化学習など他の方策最適化においても同様です．

目的関数，もしくはそれを定義する際に用いられる報酬関数$r$は，実際に獲得させたい振る舞いを反映させるように設計されることが必要です．
これは実は簡単なことではありません．
また，数学的には目的が正しく定義されていても，最適化にとって都合が悪い，という場合もあります．
そういった場合，必ずしも実際に最適化している指標が本来最適化したい指標と一致しない場合もあります．
このあたりは実用上極めて大切な部分です．
常によく検討するようにしましょう．
今回扱っている環境では，報酬自体はライブラリによって設計されていますので，この難しさは現れませんが，実用上は利用者が報酬を設計しなければならず，これがなかなか難しい場合があります．

ブラックボックス最適化法を用いて方策最適化を実現することの利点は，その汎用性です．
例えば，目的関数を比較的自由に設計することが可能です．
今回もパーセンタイルの最適化を行いましたが，これを強化学習で実現することは自明ではありません．
また，方策についても自由に選ぶことができます．
状態や行動が離散か連続かなどの違いも，方策を変更するだけで対応できます．
タスクがエピソディックタスクかどうかについても，あまり気にする必要はありません．ただし，目的関数を計算する都合上，本質的には終了状態が存在しないようなタスク（Pendulumなど）であっても，必ずどこかでエピソードを打ち切る必要は出てきます．
その点，一部の（非エピソディックなタスクに対応した）強化学習では，そのような人工的なエピソードの終了が必須ではありません．

## 自習課題

1. 目的関数を変更してみましょう．例えば，初期値分布に対して期待値を最適化するためにはどのように変更すれば良いでしょうか．
変更した目的関数を最適化した結果得られる方策を用いて複数試行回したとき，得られる報酬の期待値が他の目的関数を最適化した結果得られる値よりも小さくなっているのか，確認してみましょう．

2. 方策を変更してみましょう．例えば，Pendulumの場合，今回のニューラルネットワークを用いた方策表現では，表現能力がハイパーパラメータ$K$に依存します．これを大きくすればより柔軟な表現が可能になりますが，パラメータ数がこれに比例して大きくなります．
Lunar Landerの場合，もっと表現能力の高い方策を使用してみましょう．

3. 最適化法を変更してみましょう．ブラックボックス最適化法であれば，どのようなアプローチを用いても構いません．
方法毎に向き不向きがあります．
今回採用した(1+1)-ESは，決して方策最適化に向いているとは言い難いです．
一つの理由として，(1+1)-ESは目的関数のノイズ（目的関数の値が決定的でない）に対して適さない方法であることが挙げられます．
今回は，あくまで方策最適化というものを理解するために，単純な最適化法を採用したに過ぎません．
例えばよく知られたベイズ最適化などは，少ない目的関数の評価回数を想定して設計されているため，多くの目的関数評価を繰り返すことが想定される方策最適化には不向きとなります．
他方，強化学習では，クロスエントロピー法と呼ばれる方法が採用されることがあります．これは，別のレポジトリにて紹介しているCMA-ESに良く似た方法ですので，こちらを参考にすると良いと思います．
https://github.com/akimotolab/CMAES_Tutorial
